In this note we will take a look at the data provided for the Stochastic Modelling project. 

Following the information provided in the project documentation, we take today to be 1-Dec-2020. Let us and the load the option data and look at the data structure.

In [1]:
import datetime as dt
import pandas as pd
from pprint import pprint


spx_df = pd.read_csv('SPX_options.csv')
spx_df.head()

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style
0,20201201,20201218,C,100000,3547.6,3570.5,E
1,20201201,20201218,C,200000,3447.6,3470.5,E
2,20201201,20201218,C,300000,3347.7,3370.6,E
3,20201201,20201218,C,400000,3247.7,3270.6,E
4,20201201,20201218,C,500000,3147.7,3170.6,E


- 'date' is the current date as mentioned above
- 'exdate' is the expiry date of the option
- 'cp_flag' will be 'C' for call options and 'P' for put options
- 'strike_price' is the strike price multiplied by 1000 (*why do this?*)
- 'exercise_style' is 'E' for European, and 'A' for American

For each strike, the mid price is calculated as

\begin{equation*}
\begin{split}
\mbox{Mid Price} = \frac{\mbox{Best Bid} + \mbox{Best Ask}}{2}
\end{split}
\end{equation*}



In [2]:
spx_df['exdate'].unique()

array([20201218, 20210115, 20210219])

In [3]:
spx_df['cp_flag'].value_counts()

cp_flag
C    1036
P    1036
Name: count, dtype: int64

We can calculate the time-to-maturity $T$ in Python as follows:

In [4]:
today = dt.date(2020, 12, 1)
expiries = [pd.Timestamp(str(x)).date() for x in spx_df['exdate'].unique()]
T = [(exdate-today).days/365.0 for exdate in expiries]
pprint(dict(zip(expiries, T)))

{datetime.date(2020, 12, 18): 0.04657534246575343,
 datetime.date(2021, 1, 15): 0.1232876712328767,
 datetime.date(2021, 2, 19): 0.2191780821917808}



The file "zero_rates_20201201.csv" contains information about the "zero rates" to be used for discounting.


In [5]:
rates_df = pd.read_csv('zero_rates_20201201.csv')
rates_df.head()

,date,days,rate
0,20201201,7,0.102280
1,20201201,13,0.114128
2,20201201,49,0.216480
3,20201201,77,0.220707
4,20201201,104,0.219996


In [6]:
rates_df.tail()

,date,days,rate
40,20201201,3212,0.878441
41,20201201,3303,0.898843
42,20201201,3394,0.918827
43,20201201,3485,0.938031
44,20201201,3576,0.956515


Note that the interest rates provided in the 'rate' column are in % unit. So for instance, to discount a cashflow paid 49 days from today, the discount factor is

\begin{equation*}
\begin{split}
D(0,T) = e^{-0.00216480 \times \frac{49}{365}}
\end{split}
\end{equation*}


If the payment date is not provided in the dataframe, you can perform linear interpolation for the corresponding zero rate.